In [ ]:
!pip3 install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [ ]:
import random

import gym
from gym import Env
from gym.spaces import Box, Discrete
from gym.utils.env_checker import check_env
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

tf.compat.v1.enable_eager_execution()

In [ ]:
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))

In [ ]:
# Model Hyperparameters

STARTING_BUDGET = np.array([10.0], dtype=np.float32)
LABOR = 50
ALPHA = 0.25 # capital elasticity
DELTA = 0.25 # capital depreciation rate
EPISODE_LENGTH = 10000
BUFFER_CAPACITY = 50000
SHOCKS = False
SHOCK_TYPE = 'PERSISTENT'
DUMMY_LAG = 1 # not used for simulation; only used to initialize dummy environment

In [ ]:
class CustomEnv(Env):
    def __init__(self, lag):
        self.action_space = Box(low=0.0, high=1.0, dtype=np.float32)
        self.observation_space = Box(low=0.0, high=10000000.0, dtype=np.float32)

        self.capital = STARTING_BUDGET # the amount of capital in agent's budget
        self.capital_history = []
        self.t = 0
        self.lag = lag
        self.shock = 1
    
    def step(self, action):
        # action: float between 0.0 and 1.0, representing the savings rate for the agent
        assert(action >= 0.0 and action <= 1.0)
        assert(len(self.capital_history) == self.t)

        self.capital_history.append(self.capital)

        # update state and determine reward

        if len(self.capital_history) - self.lag < 0:
            Y = 0
        else:
            if SHOCKS:
                if SHOCK_TYPE == 'PERSISTENT'
                    self.shock = np.random.uniform(0, 2)
                    Y = self.shock * (self.capital_history[self.t - self.lag] ** ALPHA) * ((LABOR) ** (1 - ALPHA))
                elif SHOCK_TYPE == 'INTERMITTENT'
                    if len(self.capital_history) - self.lag < 0:
                        Y = 0
                    else:
                        if SHOCKS:
                            if env.t % 100 == 0:
                                self.shock = np.random.uniform(0, 2)
                            if env.t % 100 == 5:
                                self.shock = 1
            Y = self.shock * (self.capital_history[self.t - self.lag] ** ALPHA) * ((LABOR) ** (1 - ALPHA))

        self.capital = self.capital + action * Y - DELTA * self.capital
        reward = (1 - action) * Y
        self.t += 1
                
        # determine terminating condition
        if self.t >= EPISODE_LENGTH:
            done = True
        else:
            done = False
            
        info = {'production': Y,'shock': self.shock}
        
        # assert(self.capital.shape == (1,))
        
        if (type(reward) != float):
            if type(reward) == np.ndarray and len(reward) == 1:
                reward = reward[0]
            if type(reward) == np.float32:
                reward = reward.item()

        assert(type(reward) == float)

        # return step information
        return self.capital, reward, done, info
    
    def reset(self, seed=None, return_info=False, **kwargs):
        if seed:
            super().reset(seed=seed)
        
        self.capital = STARTING_BUDGET
        self.capital_history = []
        self.t = 0
        
        info = {}

        if return_info:
            return (self.capital, info)
        else:
            return self.capital
    
    def render(self):
        pass

In [ ]:
env = CustomEnv(LAG)
# check_env(env)

In [ ]:
num_states = env.observation_space.shape[0]
num_actions = env.action_space.shape[0]
upper_bound = env.action_space.high[0]
lower_bound = env.action_space.low[0]

In [ ]:
class OUActionNoise:
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None):
        self.theta = theta
        self.mean = mean
        self.std_dev = std_deviation
        self.dt = dt
        self.x_initial = x_initial
        self.reset()

    def __call__(self):
        # Formula taken from https://www.wikipedia.org/wiki/Ornstein-Uhlenbeck_process.
        x = (
            self.x_prev
            + self.theta * (self.mean - self.x_prev) * self.dt
            + self.std_dev * np.sqrt(self.dt) * np.random.normal(size=self.mean.shape)
        )
        # Store x into x_prev
        # Makes next noise dependent on current one
        self.x_prev = x
        return x

    def reset(self):
        if self.x_initial is not None:
            self.x_prev = self.x_initial
        else:
            self.x_prev = np.zeros_like(self.mean)


In [ ]:
class Buffer:
    def __init__(self, buffer_capacity=BUFFER_CAPACITY, batch_size=64):
        # Number of "experiences" to store at max
        self.buffer_capacity = buffer_capacity
        # Num of tuples to train on.
        self.batch_size = batch_size

        # Its tells us num of times record() was called.
        self.buffer_counter = 0

        # Instead of list of tuples as the exp.replay concept go
        # We use different np.arrays for each tuple element
        self.state_buffer = np.zeros((self.buffer_capacity, num_states))
        self.action_buffer = np.zeros((self.buffer_capacity, num_actions))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, num_states))

    # Takes (s,a,r,s') obervation tuple as input
    def record(self, obs_tuple):
        # Set index to zero if buffer_capacity is exceeded,
        # replacing old records
        index = self.buffer_counter % self.buffer_capacity

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]

        self.buffer_counter += 1

    # Eager execution is turned on by default in TensorFlow 2. Decorating with tf.function allows
    # TensorFlow to build a static graph out of the logic and computations in our function.
    # This provides a large speed up for blocks of code that contain many small TensorFlow operations such as this one.
    @tf.function
    def update(
        self, state_batch, action_batch, reward_batch, next_state_batch,
    ):
        with tf.device(device_name):
            # Training and updating Actor & Critic networks.
            # See Pseudo Code.
            with tf.GradientTape() as tape:
                target_actions = target_actor(next_state_batch, training=True)
                y = reward_batch + gamma * target_critic(
                    [next_state_batch, target_actions], training=True
                )
                critic_value = critic_model([state_batch, action_batch], training=True)
                critic_loss = tf.math.reduce_mean(tf.math.square(y - critic_value))

            critic_grad = tape.gradient(critic_loss, critic_model.trainable_variables)
            critic_optimizer.apply_gradients(
                zip(critic_grad, critic_model.trainable_variables)
            )

            with tf.GradientTape() as tape:
                actions = actor_model(state_batch, training=True)
                critic_value = critic_model([state_batch, actions], training=True)
                # Used `-value` as we want to maximize the value given
                # by the critic for our actions
                actor_loss = -tf.math.reduce_mean(critic_value)

            actor_grad = tape.gradient(actor_loss, actor_model.trainable_variables)
            actor_optimizer.apply_gradients(
                zip(actor_grad, actor_model.trainable_variables)
            )

    # We compute the loss and update parameters
    def learn(self):
        # Get sampling range
        record_range = min(self.buffer_counter, self.buffer_capacity)
        # Randomly sample indices
        batch_indices = np.random.choice(record_range, self.batch_size)

        # Convert to tensors
        state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])

        self.update(state_batch, action_batch, reward_batch, next_state_batch)


# This update target parameters slowly
# Based on rate `tau`, which is much less than one.
@tf.function
def update_target(target_weights, weights, tau):
    for (a, b) in zip(target_weights, weights):
        a.assign(b * tau + a * (1 - tau))

In [ ]:
def get_actor():
    # Initialize weights between -3e-3 and 3-e3
    with tf.device(device_name):
        last_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)

        inputs = layers.Input(shape=(num_states,))
        layer1 = layers.Dense(256, activation="relu")(inputs)
        outputs = layers.Dense(1, activation="sigmoid", kernel_initializer=last_init)(layer1)

        # Scaling output to fit between 0 and 1
        model = tf.keras.Model(inputs, outputs)
        model.compile()
        return model


def get_critic():
    with tf.device(device_name):
        # State as input
        state_input = layers.Input(shape=(num_states))
        state_out = layers.Dense(16, activation="relu")(state_input)

        # Action as input
        action_input = layers.Input(shape=(num_actions))
        action_out = layers.Dense(32, activation="relu")(action_input)

        # Both are passed through seperate layer before concatenating
        concat = layers.Concatenate()([state_out, action_out])

        out = layers.Dense(256, activation="relu")(concat)
        outputs = layers.Dense(1)(out)

        # Outputs single value for give state-action
        model = tf.keras.Model([state_input, action_input], outputs)
        model.compile()

        return model

In [ ]:
# def policy(state, noise_object, debugging=False):
def policy(state, noise, debugging=False):
    sampled_actions = tf.squeeze(actor_model(state))
    # Adding noise to action
    sampled_actions = sampled_actions.numpy() + noise

    # We make sure action is within bounds
    legal_action = np.clip(sampled_actions, lower_bound, upper_bound)

    if debugging:
        print("sampled_actions: {}".format(sampled_actions))
        print("np.squeeze(legal_action): {}".format(np.squeeze(legal_action)))

    return np.squeeze(legal_action)

In [ ]:
# Actor-Critic Hyperparameters
std_dev = 0.005

# Learning rate for actor-critic models
critic_lr = 0.003
actor_lr = 0.0003


# Discount factor for future rewards
gamma = 1

In [ ]:
num_trials = 30

In [ ]:
columns = ['trial', 'round', 'lag', 'savings', 'capital', 'consumption', 'noise', 'shock', 'production']
lags = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Used to update target networks
tau = 0.01

for local_lag in lags:
    env = CustomEnv(local_lag)
    for trial in range(num_trials):
        trained_data = pd.DataFrame(columns = columns) # data from the progression of the last episode

        # Reinitialize model

        ou_noise = OUActionNoise(mean=np.zeros(1), std_deviation=float(std_dev) * np.ones(1))

        actor_model = get_actor()
        critic_model = get_critic()

        target_actor = get_actor()
        target_critic = get_critic()

        # Making the weights equal initially
        target_actor.set_weights(actor_model.get_weights())
        target_critic.set_weights(critic_model.get_weights())

        critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
        actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

        buffer = Buffer(BUFFER_CAPACITY, 64)

        prev_state = env.reset()

        while True:
            tf_prev_state = tf.expand_dims(tf.convert_to_tensor(prev_state), 0)

            debugging = False

            if env.t < 500:
                action = random.uniform(0.0, 1.0)
                noise = 0
            else:
                noise = ou_noise()
                action = policy(tf_prev_state, noise, debugging=debugging)

            # Receive state and reward from environment.
            state, reward, done, info = env.step(action)

            trained_data.loc[len(trained_data.index)] = [trial, env.t, local_lag, action, state, reward, noise, info['shock'], info['production']]

            buffer.record((prev_state, action, reward, state))

            buffer.learn()
            update_target(target_actor.variables, actor_model.variables, tau)
            update_target(target_critic.variables, critic_model.variables, tau)

            if env.t % 1000 == 0:
                print("Round {} complete".format(env.t))

            # End this episode when `done` is True
            if done:
                break

            prev_state = state